In [ ]:
import pandas as pd
import sqlite3
import os

# --- CONFIG ---
file_path = r"Dummy_File_Path"   # Update as needed
sheet_name = "Tradewise Exits from 2025-04-01"

# Create SQLite database path (same directory as input file)
db_path = r"D:\TBPL.db" 

# --- LOAD SHEET WITHOUT HEADERS ---
df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# --- STEP 1: Find "F&O" section ---
fo_row = None
for idx in range(len(df)):
    if df.iloc[idx].astype(str).str.contains("F&O", case=False).any():
        fo_row = idx
        break

if fo_row is None:
    print("Could not find 'F&O' in the sheet.")
    exit()

# --- STEP 2: Find next segment marker ---
next_section_row = None
segment_markers = ["currency", "mutual fund", "mutual", "equity", "commodity"]

for idx in range(fo_row+1, len(df)):
    cell_vals = df.iloc[idx].astype(str).str.strip().str.lower()
    if any(marker in ' '.join(cell_vals) for marker in segment_markers):
        next_section_row = idx
        break

if next_section_row is None:
    next_section_row = len(df)

# --- STEP 3: Extract block between F&O and next segment ---
block = df.iloc[fo_row:next_section_row].copy()
block = block.reset_index(drop=True)

# --- STEP 4: Find the header row ("Symbol") in the block ---
header_row = None
header_col = None

for idx in range(len(block)):
    row = block.iloc[idx].astype(str).str.strip().str.lower()
    if "symbol" in row.values:
        header_row = idx
        header_col = row[row == "symbol"].index[0]
        break

if header_row is None:
    print("Could not find 'Symbol' header in F&O block.")
    exit()

# --- STEP 5: Extract and clean data table ---
data = block.iloc[header_row:, header_col:]
data.columns = data.iloc[0]
data = data[1:]  # Remove header row
data = data.dropna(how='all').reset_index(drop=True)
data = data.dropna(axis=1, how='all')

# --- STEP 6: Clean and prepare Symbol for consolidation ---
def clean_symbol(symbol):
    if pd.isna(symbol):
        return symbol
    # Convert to string, strip whitespace, and normalize case
    symbol = str(symbol).strip().upper()
    # Remove 'CE'/'PE' if present (for options)
    if symbol.endswith('CE') or symbol.endswith('PE'):
        return symbol[:-2]
    return symbol

data['Instrument'] = data['Symbol'].apply(clean_symbol)

# Debug: Print unique combinations before grouping
print("=== DEBUG: Data before grouping ===")
print("Unique Instruments:", data['Instrument'].unique())
print("Unique Exit Dates:", data['Exit Date'].unique())
print("\nData types:")
print("Instrument dtype:", data['Instrument'].dtype)
print("Exit Date dtype:", data['Exit Date'].dtype)
print("\nFirst few rows of key columns:")
print(data[['Instrument', 'Exit Date', 'Quantity']].head())

# --- STEP 7: Parse and normalize dates ---
# Convert to datetime, then normalize to remove time component, ensuring consistent grouping
data['Entry Date'] = pd.to_datetime(data['Entry Date'], errors='coerce').dt.normalize()
data['Exit Date'] = pd.to_datetime(data['Exit Date'], errors='coerce').dt.normalize()

# Debug: Check for any remaining duplicates
print("\n=== DEBUG: After date normalization ===")
print("Unique Exit Dates:", data['Exit Date'].unique())
duplicate_check = data.groupby(['Instrument', 'Exit Date']).size()
print("Groups with multiple rows:")
print(duplicate_check[duplicate_check > 1])

# --- STEP 8: Convert numeric columns to float ---
numeric_cols = ['Quantity', 'Buy Value', 'Sell Value', 'Profit', 'Turnover']
for col in numeric_cols:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')
    else:
        # If a column is not present, add it with default float values
        data[col] = 0.0

# --- STEP 9: Consolidate/group data (Entry Date: min, others: sum) ---
summary = (
    data.groupby(['Instrument', 'Exit Date'], as_index=False)
        .agg({
            'Entry Date': 'min',   # Earliest entry date for the group
            'Quantity': 'sum',
            'Buy Value': 'sum',
            'Sell Value': 'sum',
            'Profit': 'sum',
            'Turnover': 'sum'
        })
)

# --- STEP 10: Reformat date columns as DD-MM-YYYY ---
# Convert back to datetime objects if they became 'object' after aggregation
# This ensures the .dt accessor is available
summary['Entry Date'] = pd.to_datetime(summary['Entry Date']).dt.strftime('%d-%m-%Y')
summary['Exit Date'] = pd.to_datetime(summary['Exit Date']).dt.strftime('%d-%m-%Y')

# --- STEP 11: Rename 'Instrument' to 'Symbol' and set column order ---
summary = summary.rename(columns={'Instrument': 'Symbol'})
summary = summary[['Entry Date', 'Exit Date', 'Symbol', 'Quantity', 'Buy Value', 'Sell Value', 'Profit', 'Turnover']]

print("=== NEW DATA TO BE PROCESSED ===")
print(summary)

# --- SAVE TO SQLITE DATABASE WITH DUPLICATE PREVENTION ---
try:
    # Connect to SQLite database (creates file if it doesn't exist)
    conn = sqlite3.connect(db_path)
    
    # Check if table exists and load existing data
    table_exists = pd.read_sql_query(
        "SELECT name FROM sqlite_master WHERE type='table' AND name='fo_consolidated_data'",
        conn
    ).shape[0] > 0
    
    if table_exists:
        # Load existing data
        existing_data = pd.read_sql_query("SELECT * FROM fo_consolidated_data", conn)
        print(f"\n=== EXISTING DATA IN DATABASE ===")
        print(f"Number of existing records: {len(existing_data)}")
        
        # Identify duplicates based on Symbol and Exit Date
        # Create a composite key for comparison
        existing_keys = set(zip(existing_data['Symbol'], existing_data['Exit Date']))
        new_keys = set(zip(summary['Symbol'], summary['Exit Date']))
        
        # Find duplicates
        duplicate_keys = new_keys.intersection(existing_keys)
        print(f"\n=== DUPLICATE CHECK ===")
        print(f"Duplicate combinations (Symbol, Exit Date) found: {len(duplicate_keys)}")
        
        if duplicate_keys:
            print("Duplicate entries (will be skipped):")
            for symbol, exit_date in duplicate_keys:
                print(f"  - Symbol: {symbol}, Exit Date: {exit_date}")
        
        # Filter out duplicates from new data
        summary_filtered = summary[~summary.apply(
            lambda row: (row['Symbol'], row['Exit Date']) in duplicate_keys, 
            axis=1
        )]
        
        print(f"\n=== FILTERING RESULTS ===")
        print(f"Original new records: {len(summary)}")
        print(f"Records after removing duplicates: {len(summary_filtered)}")
        
        if len(summary_filtered) > 0:
            # Append only non-duplicate data
            summary_filtered.to_sql('fo_consolidated_data', conn, if_exists='append', index=False)
            print(f"\n✅ SUCCESS: {len(summary_filtered)} new records appended to database")
        else:
            print("\n⚠️  No new records to append (all were duplicates)")
            
        # Show final count
        final_count = pd.read_sql_query("SELECT COUNT(*) as count FROM fo_consolidated_data", conn).iloc[0]['count']
        print(f"Total records in database after operation: {final_count}")
        
    else:
        # Table doesn't exist, create it with all data
        summary.to_sql('fo_consolidated_data', conn, if_exists='replace', index=False)
        print(f"\n✅ SUCCESS: New table created with {len(summary)} records")

    print(f"\nDatabase path: {db_path}")
    print(f"Table name: fo_consolidated_data")

    # Close the connection
    conn.close()

except Exception as e:
    print(f"❌ Error saving to SQLite database: {e}")

# --- (Optional) Save to Excel ---
# summary.to_excel("final_consolidated_fo.xlsx", index=False)

=== DEBUG: Data before grouping ===
Unique Instruments: ['NIFTY2540323500' 'NIFTY2540923100' 'NIFTY25APR23350' 'NIFTY25APR23400'
 'NIFTY25APR23450' 'NIFTY2543024250' 'NIFTY2550824400' 'NIFTY2550824700'
 'NIFTY2551524250' 'NIFTY2551524750' 'NIFTY2552225000' 'NIFTY25MAY24550'
 'NIFTY25MAY24750' 'NIFTY2560524700' 'NIFTY2560524850' 'NIFTY2561224950'
 'NIFTY2561225000' 'NIFTY2561925250' 'NIFTY25JUN24800' 'NIFTY25JUN25000'
 'NIFTY25JUN25300' 'NIFTY2570325400' 'NIFTY2570325550' 'NIFTY2571025450'
 'NIFTY2571725300' 'NIFTY25JUL25100' 'NIFTY25JUL24900' 'NIFTY2580724900'
 'NIFTY2580724700' 'NIFTY2581424600' 'NIFTY2581424750' 'NIFTY2582125050'
 'NIFTY25AUG25100' 'NIFTY25AUG24950' 'NIFTY25AUG25000' 'NIFTY25AUG24750'
 'NIFTY2590224700' 'NIFTY2591625100' 'NIFTY2592325150' 'NIFTY25O0724650'
 'NIFTY25O0724950' 'NIFTY25O1425150' 'NIFTY25O1425400' 'NIFTY25O2025550'
 'NIFTY25OCT25900' 'NIFTY25N0426250' 'NIFTY25N0425950' 'NIFTY25N1125800'
 'NIFTY25N1825750' 'NIFTY25NOV25900' 'NIFTY25NOV26050' 'NIFTY25D0225